# Example Degurba Calculation

This sample notebook is being created in response to a question raised in [Issue #6](https://github.com/worldbank/GOSTurban/issues/6) about the Degurba Calculation functionality.

To run this notebook you will need to install the dependency [`pooch`](https://www.fatiando.org/pooch/latest/) which allows us to reference some sample data without explicitly including it in the repository.
Conveniently, `pooch` also keeps you from downloading the same data multiple times, so the first time you run the notebook the data will be downloaded and stored in a cache folder, but subsequent runs will use the cached data.

To install `pooch` you can run the following command:

```bash
pip install pooch
```

Once you have installed `pooch` you can restart the kernel and run the notebook.

## Importing the necessary libraries

In [ ]:
# import libraries
import os
import zipfile
import rasterio
import rasterio.warp

import geopandas as gpd


# Import GOSTurban functions
import GOSTurban.UrbanRaster as urban

# Import raster helpers from GOSTrocks
import GOSTrocks.rasterMisc as rMisc

# import pooch to fetch data
import pooch

## Path and Data Setup

Setting up the paths and downloading the data.

In [ ]:
# define location to put outputs
output_folder = "./output"
# make sure the folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# define files to write out
deg_file = os.path.join(output_folder, "FINAL_STANDARD", "GHS_DEG.tif")
adm1_file = os.path.join(output_folder, "ADM1.shp")

In [ ]:
# set path to global administrative boundaries file
# GEOJSON format downloaded from: https://public.opendatasoft.com/explore/dataset/world-administrative-boundaries/export/
global_bounds_adm1 = pooch.retrieve(
    url="https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/world-administrative-boundaries/exports/geojson?lang=en&timezone=America%2FNew_York",
    known_hash="a84673ed03db7196464b2771c6fd47c1afe184eb06a38c0e92cb4aedff5ee0da",
)

# define specific country of interest based on iso3 code
iso3 = "GHA"

# read the global admin boundaries
inG1 = gpd.read_file(global_bounds_adm1)
# filter the global admin boundaries to the specific country of interest
inD1 = inG1.loc[inG1["iso3"] == iso3]
# reproject the admin boundaries to WGS84
inD1 = inD1.to_crs({"init": "epsg:4326"})
# save the admin boundary for the specific country of interest to the output folder
if not os.path.exists(adm1_file):
    inD1.to_file(adm1_file)

In [ ]:
# define path to the global human settlement (GHS) population (GHS-POP) data
# can be downloaded from: https://ghsl.jrc.ec.europa.eu/download.php
pop_layer_zip = pooch.retrieve(
    url="https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_POP_GLOBE_R2023A/GHS_POP_E2030_GLOBE_R2023A_54009_100/V1-0/tiles/GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0_R9_C18.zip",
    known_hash="dba662e12455541546efdb118433fd8b08dcd36e5c978cf88e0e14bb1db599e8",
)
# unzip the GHS-POP data
zip_ref = zipfile.ZipFile(pop_layer_zip, "r")
zip_ref.extractall(output_folder)
zip_ref.close()
# path to the tif file
pop_layer = os.path.join(
    output_folder, "GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0_R9_C18.tif"
)

# file to clip data to
temp_pop_file = os.path.join(output_folder, "GHS_TEMP.tif")
# open and clip the GHS-POP data to the specific country of interest
inR = rasterio.open(pop_layer)
selD = inD1.loc[inD1["region"] == "Western Africa"]
rMisc.clipRaster(inR, selD, temp_pop_file)

## Degurba Calculation

Performing the Degurba Calculation.

In [ ]:
xx = urban.urbanGriddedPop(pop_layer)
res = xx.calculateDegurba(
    urbDens=21,
    hdDens=(15 * 7),
    minPopThresh=0.5 * 7,
    out_raster=deg_file,
    verbose=True,
)